# Result Analysis

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
%matplotlib inline

import numpy as np
np.set_printoptions(precision=2)

import pandas as pd
import pickle
import torch
from pathlib import Path
from tqdm import tqdm_notebook as tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler

from cnn.cnn_classifier.dataset import NoteDataset
from cnn.cnn_classifier.model import NoteClassifier
from cnn.args import args as cnn_args

from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pytorch_pretrained_bert.tokenization import BertTokenizer

from bert.args import args as bert_args
from bert.bert_classifier import *

from utils.embeddings import PretrainedEmbeddings
from utils.metrics import BinaryAvgMetrics
from utils.plots import *
from args import args

## 100 Run Performance Results

In [17]:
models = list(args.bc_threshold.keys())

bams = {}

for model in models:
  with open(args.workdir/model/'preds.pkl', 'rb') as f:
    targs = pickle.load(f)
    preds = pickle.load(f)
    probs = pickle.load(f)
    
  bams[model] = BinaryAvgMetrics(targs, preds, probs)

all_metrics = {}

for key in bams.keys():
  all_metrics[key] = []
  for i in range(len(bams[key].get_avg_metrics())):
    all_metrics[key].append(bams[key].get_avg_metrics().iloc[i]['Value'])

metrics = pd.DataFrame(all_metrics, index=['sensitivity', 'specificity', 'ppv', 'auroc', 'npv', 'f1'])
metrics

,lr,rf,cnn,gbm
sensitivity,0.894,0.822,0.808,0.818
specificity,0.620,0.732,0.748,0.743
ppv,0.422,0.487,0.504,0.496
auroc,0.843,0.857,0.860,0.862
npv,0.950,0.930,0.928,0.929
f1,0.573,0.612,0.617,0.617


In [12]:
bams['gbm'].get_avg_metrics(conf=0.95)

,Lower,Mean,Upper,Definition
sensitivity,0.816,0.818,0.820,"When it's ACTUALLY YES, how often does it PREDICT YES?"
specificity,0.741,0.743,0.744,"When it's ACTUALLY NO, how often does it PREDICT NO?"
ppv,0.494,0.496,0.497,"When it PREDICTS YES, how often is it correct?"
auroc,0.861,0.862,0.863,Indicates how well the model is capable of distinguishing between classes
npv,0.929,0.929,0.930,"When it PREDICTS NO, how often is it correct?"
f1,0.616,0.617,0.619,Harmonic mean of sensitivity and ppv


In [ ]:
se = (tps / (tps + fns))
ppv = (tps / (tps + fps))

In [ ]:
(2 * se * ppv)/(ppv + se)

### Model Plots

In [ ]:
model = 'gbm'
bams[model].get_avg_metrics()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_mean_roc(ax, bams[model].targs, bams[model].probs)

In [ ]:
len(bams['gbm'].targs[0])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

plot_confusion_matrix(ax[0], bams[model].cm_avg, classes=['not imminent', 'imminent'], normalize=False,\
                      title='Confusion Matrix Over Runs')
plot_confusion_matrix(ax[1], bams[model].cm_avg, classes=['not imminent', 'imminent'], normalize=True,\
                      title='Normalized Confusion Matrix Over Runs')
plt.show()

## Temporal Performance Analysis

In [ ]:
df = pd.read_csv(args.dataset_csv, parse_dates=['admittime', 'dischtime', 'intime', 'charttime'])
df['relative_charttime'] = (df['charttime'] - df['intime'])
print(df.shape)
print(df.columns)

In [ ]:
interval_hours=12
starting_day = -20
ending_day = -1

### Logistic Regression

In [ ]:
model = 'lr'

with open(args.workdir/model/'all_data_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True,\
                              ngram_range=(2,2), vocabulary=vocabulary)

df['prob'] = clf.predict_proba(vectorizer.fit_transform(df['scispacy_note']))[:, 1]

fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], starting_day, ending_day, interval_hours)

### Random Forest

In [ ]:
model = 'rf'

with open(args.workdir/model/'all_data_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True,\
                              ngram_range=(2,2), vocabulary=vocabulary)  

df['prob'] = clf.predict_proba(vectorizer.fit_transform(df['scispacy_note']))[:, 1]

fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], starting_day, ending_day, interval_hours)

### CNN

In [ ]:
model = 'cnn'

try:
  prob = np.load(cnn_args.workdir/'probs.npy')
except FileNotFoundError:
  bs = 256
  ds = NoteDataset.load_data_and_vectorizer_from_file(df, cnn_args.workdir)
  dl = DataLoader(ds, bs)
  
  pe = PretrainedEmbeddings.from_file(cnn_args.emb_path)
  pe.make_custom_embeddings(ds.vectorizer.vocab.idx_token_bidict.values())
  
  with open(args.workdir/model/'full_data_model.pkl', 'rb') as f:
    state_dict = pickle.load(f)    
  classifier = NoteClassifier(cnn_args.emb_sz, len(ds.vectorizer.vocab), cnn_args.n_channels,\
                            cnn_args.hidden_dim, 1, dropout_p=cnn_args.dropout_p,\
                            emb_dropout=cnn_args.emb_dropout, pretrained=pe.custom_embeddings)
  classifier.load_state_dict(state_dict)
  p = []

  for x, _ in tqdm(dl):
    p.append(torch.sigmoid(classifier(x)).detach().numpy())
    
  prob = np.concatenate(p)
  np.save(cnn_args.workdir/'probabilities', prob)    

df['prob'] = prob

fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], starting_day, ending_day, interval_hours)

### Gradient Boosting Machines

In [ ]:
model = 'gbm'

with open(args.workdir/model/'all_data_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True,\
                              ngram_range=(2,2), vocabulary=vocabulary)  

df['prob'] = clf.predict_proba(vectorizer.fit_transform(df['scispacy_note']))[:, 1]

fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], starting_day, ending_day, interval_hours)

### BERT

In [ ]:
model = 'bert'

try:
  features = pickle.load(open(args.workdir/model/'features.pkl', 'rb'))
except FileNotFoundError:
  tokenizer = BertTokenizer.from_pretrained(bert_args.bert_dir, do_lower_case=bert_args.do_lower_case)
  examples = []
  for i, row in df.iterrows():
    eid = f'example-{i}'
    text = row['note']
    examples.append(InputExample(eid=eid, text=text, label=-1))
  features = convert_examples_to_features(examples, bert_args.max_seq_len, tokenizer, is_pred=True)
  pickle.dump(features, open(args.workdir/model/'features.pkl', 'wb'))

In [ ]:
bert_args.bs = bert_args.bs // bert_args.gradient_accumulation_steps
clf = BertForSequenceClassification.from_pretrained(args.workdir/model, num_labels=bert_args.num_labels)
clf.to(bert_args.device)

all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)

ds = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
dl = DataLoader(ds, sampler=SequentialSampler(ds), batch_size=bert_args.bs)

In [ ]:
probs = []

for batch in tqdm(dl, desc='Running model'):
  batch = tuple(t.to(bert_args.device) for t in batch)
  input_ids, input_mask, segment_ids = batch
  logits = clf(input_ids, input_mask, segment_ids)
  prob = torch.sigmoid(logits).detach().cpu().numpy()
  if len(probs) == 0:
    probs.append(prob)
  else:
    probs[0] = np.append(
      probs[0], prob, axis=0)

df['prob'] = np.squeeze(probs[0])  

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], cutoff=cutoff, interval=interval, granularity=2)

In [ ]:
with open(args.workdir/model/'all_data_model.pkl', 'rb') as f:
  clf = pickle.load(f)
  vocabulary = pickle.load(f)
  
vectorizer = TfidfVectorizer(min_df=3, analyzer=str.split, sublinear_tf=True,\
                              ngram_range=(2,2), vocabulary=vocabulary)  

df['prob'] = clf.predict_proba(vectorizer.fit_transform(df['scispacy_note']))[:, 1]

fig, ax = plt.subplots(figsize=(15, 8))
plot_prob(ax, df, args.bc_threshold[model], cutoff=cutoff, interval=interval, granularity=2)